# 🛡️ Safety Text Classifier Training - Google Colab

**Constitutional AI Research Project - Stage 1**

This notebook trains a transformer-based safety text classifier using JAX/Flax on Google Colab GPU.

## 📋 Prerequisites
- GPU runtime enabled in Colab
- Weights & Biases account for experiment tracking
- Project files uploaded or cloned

## 🎯 Expected Results
- Training time: ~2-3 hours on GPU
- Target accuracy: 85%+ on safety classification
- Model size: ~50MB

## 1. 🚀 Environment Setup

In [ ]:
# Check GPU availability
!nvidia-smi
print("\n" + "="*50)
print("🎯 If you see GPU info above, you're ready!")
print("❌ If not, go to Runtime -> Change runtime type -> GPU")
print("="*50)

In [ ]:
# Mount Google Drive for checkpoints
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint directory on Drive
import os
checkpoint_dir = '/content/drive/MyDrive/safety-classifier-checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
print(f"✅ Checkpoint directory ready: {checkpoint_dir}")

In [ ]:
# Upload project files (choose one method)
print("📁 Upload your project files using one of these methods:")
print("1. Zip your project and upload via file browser")
print("2. Clone from GitHub (recommended - uncomment below)")
print("3. Copy from Google Drive (uncomment below)")
print()

# Uncomment ONE of the following methods:

# Method 1: GitHub clone (ml-learning repo)
# !git clone https://github.com/yourusername/ml-learning.git
# %cd ml-learning/safety-text-classifier

# Method 2: Google Drive copy (ml-learning repo)
# !cp -r "/content/drive/MyDrive/ml-learning" .
# %cd ml-learning/safety-text-classifier

# Method 3: Manual upload (use file browser on left)
# If you upload ml-learning.zip, extract and navigate:
# !unzip ml-learning.zip
# %cd ml-learning/safety-text-classifier

# Verify project structure
!ls -la
print("\n✅ Make sure you see: src/, configs/, requirements.txt")
print("\n📁 Current directory:")
import os
print(os.getcwd())

## 1.5 🚀 Modern Setup with uv (OPTIONAL)

This step installs dependencies using uv (10-100x faster than pip) with proper compatibility.

**Skip this section if you want to use the automated setup below.**

In [ ]:
# OPTIONAL: Manual uv installation and setup
# This is faster but requires more steps

# Install uv package manager
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Add uv to PATH
import os
uv_path = os.path.expanduser('~/.cargo/bin')
os.environ['PATH'] = f"{uv_path}:{os.environ['PATH']}"

# Test uv
!uv --version

In [ ]:
# OPTIONAL: Install dependencies with uv using requirements-colab.txt
# Only run this if you ran the uv installation above

# Install base packages
!uv pip install -r requirements-colab.txt

# Install GPU JAX separately
!uv pip install jax[cuda12]==0.4.28 --find-links https://storage.googleapis.com/jax-releases/jax_cuda_releases.html --force-reinstall

print('✅ uv installation complete!')

## 1.9 📍 Navigate to Project Directory

This cell helps you navigate to the correct directory if you've uploaded the full ml-learning repository.

In [ ]:
# Navigation Helper - Run this if you have path issues
import os
from pathlib import Path

def find_and_navigate_to_project():
    current_dir = Path.cwd()
    print(f"📍 Current directory: {current_dir}")
    
    # Check if we're already in the right place
    if current_dir.name == "safety-text-classifier":
        if (current_dir / "src").exists() and (current_dir / "configs").exists():
            print("✅ Already in safety-text-classifier directory!")
            return str(current_dir)
    
    # Look for safety-text-classifier in current directory
    safety_path = current_dir / "safety-text-classifier"
    if safety_path.exists() and safety_path.is_dir():
        print(f"✅ Found safety-text-classifier at: {safety_path}")
        os.chdir(safety_path)
        print(f"📂 Changed to: {Path.cwd()}")
        return str(safety_path)
    
    # Look for ml-learning directory
    ml_learning_path = current_dir / "ml-learning"
    if ml_learning_path.exists():
        safety_in_ml = ml_learning_path / "safety-text-classifier"
        if safety_in_ml.exists():
            print(f"✅ Found project at: {safety_in_ml}")
            os.chdir(safety_in_ml)
            print(f"📂 Changed to: {Path.cwd()}")
            return str(safety_in_ml)
    
    print("❌ Could not find safety-text-classifier directory!")
    print("📋 Available directories:")
    for item in current_dir.iterdir():
        if item.is_dir():
            print(f"  📁 {item.name}")
    return None

# Run navigation
project_path = find_and_navigate_to_project()

# Verify structure
if project_path:
    print("\n🔍 Verifying project structure...")
    required_items = ['src', 'configs', 'requirements.txt']
    all_good = True
    for item in required_items:
        if Path(item).exists():
            print(f"✅ {item}")
        else:
            print(f"❌ {item}")
            all_good = False
    
    if all_good:
        print("\n🎉 Project structure looks good!")
    else:
        print("\n⚠️  Some required files are missing.")

print(f"\n📍 Final directory: {Path.cwd()}")

In [ ]:
# Run the automated setup (now with smart path detection)
import sys

# Add current directory to path
sys.path.append(str(Path.cwd()))

# Run setup script with error handling
try:
    exec(open('src/colab_setup.py').read())
    print("\n🎯 Setup complete! Ready for training.")
except FileNotFoundError as e:
    print(f"❌ Setup script not found: {e}")
    print(f"📍 Current directory: {Path.cwd()}")
    print("\n📋 Available files:")
    for item in Path.cwd().iterdir():
        print(f"  {item.name}")
    print("\n💡 Please run the navigation cell above first!")

## 2. 🏋️ Model Training

In [ ]:
# Import required modules - Fixed version
# This should now work without numpy compatibility issues

# Verify numpy is correctly installed first
import numpy as np
print(f"Numpy version: {np.__version__}")

# Now import JAX and other dependencies
import jax
import jax.numpy as jnp
import sys
import os
from pathlib import Path

# Add src to path
sys.path.append('src')

# Try importing trainer - this should work now
from training.trainer import SafetyTrainer

# Verify JAX setup
print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")
print(f"JAX backend: {jax.default_backend()}")

# Check for GPU
gpu_devices = [d for d in jax.devices() if 'gpu' in str(d).lower()]
if gpu_devices:
    print(f"🎯 GPU ready for training: {gpu_devices}")
else:
    print("⚠️  No GPU detected - training will be slow!")

In [ ]:
# Setup Weights & Biases
import wandb

# Login to W&B (you'll need to paste your API key)
wandb.login()

print("✅ W&B setup complete!")

In [ ]:
# Start training with Colab-optimized config
print("🚀 Starting training with Colab configuration...")
print("📊 Monitor progress in W&B dashboard")
print("⏱️  Expected training time: 2-3 hours")
print("="*60)

try:
    # Initialize trainer with Colab config
    trainer = SafetyTrainer(config_path="configs/colab_config.yaml")
    
    # Start training
    trainer.train()
    
    print("\n🎉 Training completed successfully!")
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    import traceback
    traceback.print_exc()

finally:
    # Clean up W&B
    if 'wandb' in globals() and wandb.run:
        wandb.finish()

## 3. 📊 Model Evaluation

In [ ]:
# Evaluate the trained model
import sys
sys.path.append('src')

from training.trainer import SafetyTrainer
from data.dataset_loader import create_data_loaders
import yaml
import numpy as np

print("📊 Loading model for evaluation...")

# Load config
with open('configs/colab_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Load trainer (will load best checkpoint)
trainer = SafetyTrainer('configs/colab_config.yaml')

# Load test data
_, _, test_dataset = create_data_loaders('configs/colab_config.yaml')

# Evaluate on test set
test_metrics = trainer.evaluate(test_dataset, 0, "test/")

print("\n🎯 Final Test Results:")
print(f"Test Accuracy: {test_metrics['accuracy']:.1%}")
print(f"Test Loss: {test_metrics['loss']:.4f}")

# Per-class accuracy
if 'per_class_accuracy' in test_metrics:
    categories = ['Hate Speech', 'Self Harm', 'Dangerous Advice', 'Harassment']
    per_class_acc = test_metrics['per_class_accuracy']
    print("\n📋 Per-Category Accuracy:")
    for cat, acc in zip(categories, per_class_acc):
        print(f"  {cat}: {acc:.1%}")

## 4. 🧪 Interactive Demo

In [ ]:
# Run interactive demo with trained model
import sys
sys.path.append('src')

from models.transformer import create_model, initialize_model
from models.utils import count_parameters
import yaml
import jax
import jax.numpy as jnp
from transformers import AutoTokenizer

print("🧪 Setting up interactive demo...")

# Load config and model
with open('configs/colab_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

model = create_model(config)

# Initialize for demo (you'd load actual trained weights)
rng = jax.random.PRNGKey(42)
params = initialize_model(model, rng)

# Setup tokenizer
tokenizer = AutoTokenizer.from_pretrained(config['data']['tokenizer'])
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def classify_text(text: str):
    """Classify a text sample."""
    # Tokenize
    inputs = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=config['data']['max_length'],
        return_tensors='np'
    )
    input_ids = jnp.array(inputs['input_ids'])
    
    # Get predictions
    outputs = model.apply(params, input_ids, training=False)
    logits = outputs['logits']
    probabilities = jax.nn.sigmoid(logits)[0]  # Remove batch dim
    
    # Category mapping
    categories = {
        0: 'Hate Speech',
        1: 'Self Harm',
        2: 'Dangerous Advice',
        3: 'Harassment'
    }
    
    # Create results
    results = {}
    for i, prob in enumerate(probabilities):
        results[categories[i]] = float(prob)
    
    # Determine overall safety
    max_prob = max(results.values())
    if max_prob > 0.5:
        safety_status = f"⚠️  POTENTIALLY UNSAFE ({max(results, key=results.get)})"
    else:
        safety_status = "✅ SAFE"
    
    return results, safety_status

# Test examples
test_examples = [
    "The weather is beautiful today!",
    "I hate all people from that country",
    "How to make dangerous explosives at home",
    "I'm going to find you and hurt you"
]

print("\n🎯 Testing classifier on sample texts:\n")

for i, example in enumerate(test_examples, 1):
    results, status = classify_text(example)
    print(f"Example {i}: {example[:50]}...")
    print(f"Status: {status}")
    print("Scores:")
    for category, score in results.items():
        print(f"  {category}: {score:.1%}")
    print(""+"-"*50)

## 5. 💾 Save Model for Production

In [ ]:
# Save trained model to Google Drive for later use
import shutil
from pathlib import Path

print("💾 Saving model to Google Drive...")

# Create Drive directory
drive_model_dir = '/content/drive/MyDrive/safety-classifier-model'
Path(drive_model_dir).mkdir(parents=True, exist_ok=True)

# Copy best model checkpoint
if Path('checkpoints/best_model').exists():
    shutil.copytree('checkpoints/best_model', f'{drive_model_dir}/best_model', dirs_exist_ok=True)
    print("✅ Best model saved to Drive")

# Copy configuration
shutil.copy('configs/colab_config.yaml', f'{drive_model_dir}/config.yaml')
print("✅ Configuration saved to Drive")

# Copy training logs
if Path('logs').exists():
    shutil.copytree('logs', f'{drive_model_dir}/logs', dirs_exist_ok=True)
    print("✅ Logs saved to Drive")

# Create model info file
model_info = f"""
Safety Text Classifier - Trained Model
=====================================

Model Architecture: Transformer-based (JAX/Flax)
Parameters: ~67M
Training Framework: Constitutional AI Research Pipeline

Safety Categories:
- Hate Speech
- Self Harm
- Dangerous Advice
- Harassment

Files:
- best_model/: Model checkpoint
- config.yaml: Model configuration
- logs/: Training logs

Usage:
Load this model in your safety text classifier implementation.
"""

with open(f'{drive_model_dir}/README.txt', 'w') as f:
    f.write(model_info)

print(f"\n🎉 Model successfully saved to Google Drive!")
print(f"📁 Location: {drive_model_dir}")
print("\n🚀 Your Safety Text Classifier is ready for deployment!")

## 🎯 Next Steps: Constitutional AI Pipeline

Congratulations! You've completed **Stage 1** of the Constitutional AI research pipeline.

### ✅ Stage 1 Complete: Safety Text Classifier
- ✅ Transformer model trained
- ✅ 85%+ accuracy achieved
- ✅ Multi-category safety detection
- ✅ Model saved and ready for use

### 🔄 Coming Next: Stage 2 - Helpful Response Fine-tuning

Your next project will be:
- **Goal**: Fine-tune Gemma 7B-IT for helpful behavior
- **Duration**: Month 3-4
- **Skills**: Transfer learning, fine-tuning, behavior shaping
- **Foundation**: This safety classifier will evaluate the fine-tuned model

### 📚 What You've Learned

1. **JAX/Flax**: Functional neural network programming
2. **Transformer Architecture**: Multi-head attention, positional encoding
3. **Safety Research**: Multi-category harm detection
4. **MLOps**: Training pipelines, experiment tracking, model deployment
5. **Constitutional AI**: Foundation for alignment research

### 🏆 Portfolio Impact

This project demonstrates:
- **Technical Skills**: Modern ML frameworks, research implementation
- **AI Safety**: Understanding of harmful content detection
- **Research Methodology**: Systematic approach to alignment problems
- **Production Ready**: Complete training and deployment pipeline

**🎉 Great work! You're ready for Stage 2 of Constitutional AI research!**